In [102]:
import pandas as pd
import json
import nltk

In [103]:
!pip install pyvis
!pip install stop-words
!pip install pymorphy3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [104]:
df = pd.read_csv('russian_demographics.csv', converters={'Скачанные данные': eval})
df.head(10)
df["Скачанные данные"][0].keys()

dict_keys(['Введение', 'Физико-географическая характеристика', 'География', 'Природные ресурсы', 'Часовой пояс', 'Климат', 'История', 'Население', 'Национальный состав', 'Здравоохранение', 'Населённые пункты', 'Административно-территориальное деление', 'Телефонные коды муниципальных образований', 'Суды', 'Экономика', 'Сельское хозяйство', 'Машиностроение', 'Энергетика', 'Транспорт', 'Туризм', 'СМИ', 'Религии', 'Наука, образование и культура', 'Театры и концертные организации', 'Спорт', 'См. также', 'Примечания', 'Литература', 'Ссылки'])

In [105]:
subjects = []
for row in df.iloc:
    subjects.append([row['Субъект Российской Федерации'], row['Скачанные данные']])
print(subjects[0])

['Республика Адыгея', {'Введение': 'Адыге́я или Респу́блика Адыге́я (адыг. Адыгэ Республик) — субъект Российской Федерации, республика в её составе.\nВходит в состав Южного федерального округа, является частью Северо-Кавказского экономического района. Столица — город Майкоп.\nПредставляет собой анклав: территория республики со всех сторон окружена территорией Краснодарского края.\nОбразована 27 июля 1922 года как Черкесская (Адыгейская) автономная область на территории Майкопского и Екатеринодарского отделов Кубано-Черноморской области, 24 августа 1922 года Черкесская (Адыгейская) автономная область была переименована в Адыгейскую (Черкесскую) автономную область. В августе 1928 года была переименована в Адыгейскую автономную область, 3 июля 1991 года преобразована в Советскую Социалистическую Республику Адыгея. 23 марта 1992 года принято современное название.\nГосударственные языки: адыгейский и русский.\n\n\n', 'Физико-географическая характеристика': '\n\n', 'География': '\nРеспублика

In [106]:
from sentence_transformers import SentenceTransformer

# Загрузка модели, поддерживающей русский язык
model = SentenceTransformer('cointegrated/rubert-tiny')

# Пример списка пар (название, описание)

# Извлекаем описания
descriptions = [desc for _, desc in subjects]

# Получаем эмбеддинги для описаний
embeddings = model.encode(descriptions, show_progress_bar=True)


No sentence-transformers model found with name cointegrated/rubert-tiny. Creating a new one with mean pooling.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [107]:
embeddings

array([[ 1.2038959 , -0.10415141,  0.20244767, ..., -0.02641412,
         0.5124455 , -0.8782423 ],
       [ 1.1452742 ,  0.01755611,  0.28043187, ...,  0.09788644,
         0.4239082 , -0.90722185],
       [ 0.9234046 , -0.12771751, -0.14307934, ...,  0.11529087,
         0.14603913, -0.8971127 ],
       ...,
       [ 1.1818931 ,  0.08698319, -0.2122657 , ...,  0.13328145,
         0.62014866, -1.2895395 ],
       [ 1.2914025 ,  0.25353563,  0.19263498, ...,  0.03653248,
         0.5030401 , -0.8296744 ],
       [ 0.9747609 ,  0.11785842, -0.08082044, ...,  0.03049682,
         0.2946202 , -0.87066096]], shape=(89, 312), dtype=float32)

In [108]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# Расчёт матрицы косинусного сходства
sim_matrix = cosine_similarity(embeddings)

# Создаём граф, добавляем вершины и ребра с весами сходства
G = nx.Graph()
for i in range(len(subjects)):
    name = subjects[i][0]
    G.add_node(i, label=name)

for i in range(len(embeddings)):
    for j in range(i+1, len(embeddings)):
        weight = sim_matrix[i][j]
        if weight > 0.95:  # например, порог сходства для добавления ребра
            G.add_edge(i, j, weight=weight)
print([len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)])
print(list(nx.connected_components(G)))

[35, 10, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[{0, 1, 4, 7, 9, 17, 19, 20, 21, 23}, {2}, {3, 25, 29, 32, 33, 35, 37, 39, 41, 44, 45, 47, 48, 50, 51, 52, 54, 55, 57, 58, 60, 62, 63, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 77, 79}, {13, 5}, {8, 26, 6}, {10}, {11}, {12}, {14}, {15}, {16}, {18}, {22}, {24}, {27}, {28}, {30}, {31}, {34}, {36}, {80, 38}, {40}, {42, 78}, {43}, {46}, {49}, {53}, {56}, {59}, {61}, {64}, {69}, {76}, {81, 82}, {83}, {84}, {88, 85, 86}, {87}]


In [110]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize(text):
    return ' '.join([morph.parse(word)[0].normal_form for word in text.split()])

russian_stopwords = get_stop_words("russian")
# Преобразуем descriptions в список текстов
texts = [lemmatize(str(desc)) for desc in descriptions]

# Инициализируем TF-IDF векторизатор
vectorizer = TfidfVectorizer(stop_words=russian_stopwords, max_features=1000)  # можно подстроить параметры
tfidf_matrix = vectorizer.fit_transform(texts)
feature_names = vectorizer.get_feature_names_out()

# Для каждой компоненты находим топ-N слов
top_n = 10
for idx, component in enumerate(nx.connected_components(G)):
    indices = list(component)
    component_tfidf = tfidf_matrix[indices]  # вытаскиваем подматрицу
    mean_tfidf = np.asarray(component_tfidf.mean(axis=0)).flatten()

    top_indices = mean_tfidf.argsort()[::-1][:top_n]
    top_words = [feature_names[i] for i in top_indices]

    print(f"Component {idx+1} (size {len(indices)}):")
    print("Top words:", top_words)
    print()


Component 1 (size 10):
Top words: ['республика', 'республики', 'тыс', 'хакасия', 'дагестан', 'адыгея', 'чувашский', 'алтай', 'nть', 'тыва']

Component 2 (size 1):
Top words: ['башкортостан', 'башкирский', 'республика', 'nть', 'советский', 'республики', 'территория', 'флаг', 'конституция', 'являться']

Component 3 (size 35):
Top words: ['область', 'области', 'тыс', 'nть', 'территория', 'район', 'га', 'население', 'производство', 'состав']

Component 4 (size 2):
Top words: ['лнр', 'днр', 'донецкий', 'луганский', 'украинский', 'народный', 'украина', '2014', 'украины', 'республика']

Component 5 (size 3):
Top words: ['ингушетия', 'камчатский', 'калмыкия', 'калмыцкий', 'республика', 'ингушский', 'nть', 'территория', 'тыс', 'край']

Component 6 (size 1):
Top words: ['карелия', 'карельский', 'республика', 'nть', 'язык', 'гэс', 'государственный', 'союз', 'россия', 'республики']

Component 7 (size 1):
Top words: ['коми', 'республика', 'км', 'усть', 'река', 'урал', 'озеро', 'район', 'тыс', 'севе

In [74]:
for comp in nx.connected_components(G):
    s = set()
    for i in comp:
        print(descriptions[i][0])
        break
    print(comp)


KeyError: 0

In [24]:
from pyvis.network import Network

net = Network(height='750px', width='100%', notebook=True)

# Добавляем узлы
for node, data in G.nodes(data=True):
    net.add_node(node, label=data.get('label', str(node)))

# Добавляем ребра, приводя вес к типу float
for source, target, data in G.edges(data=True):
    weight = float(data.get('weight', 1))
    net.add_edge(source, target, value=weight)

net.force_atlas_2based()
net.show('graph4.html')


graph4.html
